# 1. 앙상블
- 약한 학습기를 각기 다른 알고리즘으로 학습시켜 최종결과 voting
- 큰 수의 법칙
- 각 모델이 독립적이어야 효과 좋음

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=500, noise=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

voting_clf= VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [2]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred= clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8466666666666667
RandomForestClassifier 0.8333333333333334
SVC 0.9
VotingClassifier 0.8733333333333333


# 2. Bagging, pasting, OOB
- sampling 된지않은 데이터 oob set을 성능평가에 활용

In [3]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf= BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, oob_score=True
)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
bag_clf.oob_score_
# bag_clf.oob_decision_function_

0.92

# 3. Random Forest, Extra Tree
- 배깅/페이스팅을 적용한 결정트리 앙상블
- 트리의 노드를 분할할 때 
    - 의사결정트리: 전체 특성 중 최선의 특징으로 나눈다.
    - 랜덤 포레스트
        - 부트스트랩 샘플(중복된 훈련 샘플)을 사용
        - 주어진 모든 feature에 대한 정보이득을 계산, 가장 높은 정보 이득을 가지는 feature들을 전부 비교해서 가장 최선의 feature를 선정
    - 엑스트라 트리
        - 훈련 세트 전체를 사용(배깅은 아님)
        - Split을 할 때 무작위로 feature를 선정. feature중에 아무거나 고른 다음 그 feature에 대해서 최적의 Node를 분할. 
- 특성 중요도
    - 평균적으로 불순도를 얼마나 감소시키는지 합이 1이 되도록 정규화

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

rnd_clf=RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
bag_clf=BaggingClassifier(DecisionTreeClassifier(max_features="sqrt", max_leaf_nodes=16),n_estimators=500)

In [5]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09874311436055545
sepal width (cm) 0.0245208328960225
petal length (cm) 0.44595187162369837
petal width (cm) 0.43078418111972355


# 4. AdaBoost
- 이전 예측기가 과소적합했던 훈련 샘플 가중치를 상대적으로 더 높임
- 학습하기 어려운 샘플에 맞춰짐

In [6]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200,
    algorithm="SAMME.R",
    learning_rate=0.5
)

ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

# 5. Gradient Boosting
- 샘플의 가중치 수정하는 대신 이전 예측기가 만든 잔여오차에 새로운 예측기 학습
- 예측기 수가 부족하거나 많으면 과소적합, 과대적합 발생
    - 최적의 트리 개수 찾기위해 조기종료 사용
- 학습률(각 트리의 기여정도)을 조절하여 규제 가능
- 확률적 그래디언트 부스팅
    - 각 트리에서 훈련에 사용하는 데이터 샘플 비율 설정(subsample)
    - 훈련속도 개선, 규제효과
- 유명 모델: XGBoost

In [11]:
# 원리
from sklearn.tree import DecisionTreeRegressor

tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X_train,y_train)

y2=y_train-tree_reg1.predict(X_train)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X_train,y2)

y3=y2-tree_reg1.predict(X_train)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X_train,y3)

y_pred= sum(tree.predict(X_test) for tree in (tree_reg1, tree_reg2, tree_reg3 ))

In [13]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [15]:
# 조기 종료 적용
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

grbt= GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error=float('inf')
error_goning_up= 0
for n_estimators in range(1, 120):
    grbt.n_estimators= n_estimators
    grbt.fit(X_train, y_train)
    y_pred= grbt.predict(X_val)
    val_error= mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error= val_error
        error_going_up= 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break

In [16]:
# xgboost
import xgboost

xgb_reg= xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train, evel_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred= xgb_reg.predict(X_val)

ModuleNotFoundError: No module named 'xgboost'

# 6.Stacking
- Stacked Generalization
- 예측 취합 함수 대신 예측 취합 모델(블렌더) 생성
    - 훈련셋을 서브셋1과 서브셋2(hold-out)으로 나눈다.
    - 서브셋1로 예측기들 훈련
    - 서브셋2로 훈련된 예측기들의 예측결과 생성
    - 서브셋2 예측결과들을 특성값으로 하여 블렌더가 클래스값 맞추도록 학습